# **Instalando os pacotes necessários**
#### `Antes de utilizar este notebook lembre de baixar o Julia e o Jupyter corretamente em sua máquina`
<div class="alert alert-block alert-info">
<b>!!!</b> Antes de utilizar este notebook lembre de baixar o Julia e o Jupyter corretamente em sua máquina
</div>

***

`Testanto sintaxe do markdowns`

$Testantosintaxedomarkdowns$ 

$v \cdot u = v_{1}u_{1} + v_{2}u_{2}$
> Testando sintaxe do markdown
- Testando sintaxe do markdown
-  Testando sintaxe do markdown
* Testando sintaxe do markdown

***

- Testando sintaxe do markdown
    - testando sintaxe do markdown

***

1. Testando sintaxe
2. Testando sintaxe 
3. etc...

***

<div class="alert alert-block alert-danger">
<b>Testando a sintaxe do markdown</b> Testando a sintaxe do markdown
</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not 
inside code cells, or use for mathematical formulas if needed.
</div>

In [1]:
#Execute este bloco de código apenas uma vez

using Pkg

#digitando comentario

# Instalar pacotes
Pkg.add("HTTP")
Pkg.add("JSON")
Pkg.add("IJulia")

    Updating registry at `C:\Users\Gorguet\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Gorguet\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Gorguet\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Gorguet\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Gorguet\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Gorguet\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Gorguet\.julia\environments\v1.11\Manifest.toml`


# **Definindos Funções**
 Estamos utilizando aqui os conceitos que aprendemos em Algebra linear Algorítmica para trabalhar com vetores e entender o quão próximos dois vetores estão.


In [ ]:
using LinearAlgebra

function cos_calc(v1, v2)

  return (dot(v1,v2)/(norm(v1)*norm(v2)))

end

function calc_angle(v1, v2)
  return (((180)/pi)*(acos(cos_calc(v1, v2))))

end

function mult_matrix(v1, m)
  

end

v1 = [1,0,0]
v2 = [0,1,0]

v3 = transpose(v2)

println(v2)
println(v3)

println(cos_calc(v1, v2))
println(calc_angle(v1, v2))


[0, 1, 0]
[0 1 0]
0.0
90.0
